In [14]:
import tensorflow as tf
import keras
from tensorflow.keras.initializers import RandomNormal

In [1]:
def load_images(path, size=(256,512)):
    src_img, targ_img = list(), list()
    for filename in listdir(path):
        image = load_img(path+'/'+filename, target_size=size)
        pixels = img_to_array(image)
        edge_image, real_image = pixels[:,:256], pixels[:,256:]
        src_img.append(edge_image) 
        targ_img.append(real_image)
    return [asarray(src_img), asarray(targ_img)]

In [2]:
def discriminator(image_shape):
    init = RandomNormal(stddev=0.02)
    in_src_image = keras.layers.Input(shape=image_shape)
    in_target_image = keras.layers.Input(shape=image_shape)
    merged = keras.layers.Concatenate()([in_src_image, in_target_image])
    d = keras.layers.Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    d = keras.layers.Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = keras.layers.BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = keras.layers.Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = keras.layers.BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = keras.layers.Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = keras.layers.BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = keras.layers.Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = keras.layers.BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = keras.layers.Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = keras.layers.Activation('sigmoid')(d)
    model = keras.models.Model([in_src_image, in_target_image], patch_out)
    opt = keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [3]:
def encoder_block(input_tensor, n_filters, kernel_size = 4,      batchnorm=True):
    init = RandomNormal(stddev=0.02)
    # first layer
    x = keras.layers.Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size), strides=(2,2), padding = 'same', kernel_initializer=init)(input_tensor)
    if batchnorm:
        x = keras.layers.BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    return x
def decoder_block(input_tensor, skip_layer, n_filters, dropout=True):
    init = RandomNormal(stddev=0.02)
    x = keras.layers.Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding="same", kernel_initializer=init)(input_tensor)
    x = keras.layers.BatchNormalization()(x, training=True)
    
    if dropout:
        x = keras.layers.Dropout(0.5)(x, training=True)
    x = keras.layers.Concatenate()([x, skip_layer])
    x = keras.layers.Activation("relu")(x)
    return x
def generator(input_img=(256,256,3), batchnorm=True, dropout=True):
    init = RandomNormal(stddev=0.02)
    image = keras.layers.Input(shape=input_img)
    # Contracting Path
    c1 = encoder_block(image, 64, kernel_size = 4, batchnorm=False)
    c2 = encoder_block(c1, 128, kernel_size = 4, batchnorm=batchnorm)
    c3 = encoder_block(c2, 256, kernel_size = 4, batchnorm=batchnorm)
    c4 = encoder_block(c3, 512, kernel_size = 4, batchnorm=batchnorm)
    c5 = encoder_block(c4, 512, kernel_size = 4, batchnorm=batchnorm)
    c6 = encoder_block(c5, 512, kernel_size = 4, batchnorm=batchnorm)
    c7 = encoder_block(c6, 512, kernel_size = 4, batchnorm=batchnorm)
    m = keras.layers.Conv2D(512, (4,4), strides = (2,2), padding="same", kernel_initializer=init)(c7)
    m = keras.layers.Activation("relu")(m)
    # Expansive Path
    d1 = decoder_block(m, c7, 512, dropout=dropout)
    d2 = decoder_block(d1, c6, 512, dropout=dropout)
    d3 = decoder_block(d2, c5, 512, dropout=dropout)
    d4 = decoder_block(d3, c4, 512, dropout=False)
    d5 = decoder_block(d4, c3, 256, dropout=False)
    d6 = decoder_block(d5, c2, 128, dropout=False)
    d7 = decoder_block(d6, c1, 64, dropout=False)
    outputs = keras.layers.Conv2DTranspose(3, (4, 4), strides=(2,2), padding="same", kernel_initializer=init)(d7)
    outputs = keras.layers.Activation("tanh")(outputs)
    model = keras.models.Model(inputs=[image], outputs=[outputs])
    return model

In [4]:
def gan_model(generator, discriminator, input_img):
    discriminator.trainable = False
    src_input = keras.layers.Input(shape = input_img)
    gen_output = generator(src_input)
    disc_output = discriminator([src_input, gen_output])
    model = keras.models.Model(inputs=src_input, outputs=[disc_output, gen_output])
    opt = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss=["binary_crossentropy", "mae"], optimizer=opt, loss_weights=[1,100])
    return model

In [15]:
discriminator = discriminator((178, 218))

ValueError: Input 0 of layer conv2d is incompatible with the layer: : expected min_ndim=4, found ndim=3. Full shape received: (None, 178, 436)